In [1]:
from datasets import load_dataset

ds = load_dataset("wikimedia/wikipedia", "20231101.en")

c:\Users\Profe\.virtualenvs\code-uVaVH2Jt\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
from qdrant_client import QdrantClient
from qdrant_client import models
import numpy as np


c:\Users\Profe\.virtualenvs\code-uVaVH2Jt\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import random

# Получаем количество элементов в наборе данных
num_samples = len(ds['train'])

# Генерируем случайный индекс
random_index = random.randint(0, num_samples - 1)

# Извлекаем случайный элемент
random_sample = ds['train'][random_index]

# Печатаем случайный объект
print(random_sample)

{'id': '36414119', 'url': 'https://en.wikipedia.org/wiki/Harry%20Wrathall', 'title': 'Harry Wrathall', 'text': "Harry Wrathall (1 February 1869 – 1 June 1944) was an English cricketer who played for Gloucestershire, Marylebone and London County.\n\nCareer\nBetween 1894 and 1907, Wrathall played in 288 first-class matches. The right handed batsman played 509 innings with an average of 22.54. Wrathall's right-arm medium pace bowling took 30 wickets averaging at 45.26.\n\nReferences\n\n1869 births\n1944 deaths\nEnglish cricketers of 1890 to 1918\nEnglish cricketers\nCricketers from Cheltenham\nGloucestershire cricketers\nLondon County cricketers\nMarylebone Cricket Club cricketers\nPlayers cricketers\nMidland Counties cricketers\nNorth v South cricketers\nNorthumberland cricketers\nPlayers of the South cricketers\nCricketers from Gloucestershire"}


In [2]:
import os

client = QdrantClient(path='rag_database')
COLLECTION_NAME = 'wikipedia_20231101_en_text512_embeddings'


In [3]:
from qdrant_client.models import ScalarQuantization, ScalarQuantizationConfig
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
'''
# Получение списка всех коллекций
collections = client.get_collections().collections

# Удаление всех коллекций
for collection in collections:
    client.delete_collection(collection.name)
    print(f"Коллекция '{collection.name}' удалена")
'''
client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(
                            size=1024,
                            distance=Distance.COSINE,
                            on_disk=True,
                            #quantization_config=models.ScalarQuantization(
                            #scalar=models.ScalarQuantizationConfig(
                            #    type=models.ScalarType.INT8,
                            #),
                            #)
                        ),
    optimizers_config=models.OptimizersConfigDiff(memmap_threshold=20000),
    hnsw_config=models.HnswConfigDiff(on_disk=True),
    on_disk_payload=True,

)


True

In [5]:
# Let's try opening the specified file and print the first 10 lines.

# Define the file path
file_path = 'C:/Users/Profe/OneDrive/Документы/embeddings_part1.json'

# Reading the first 10 lines of the file
try:
    with open(file_path, 'r') as file:
        first_10_lines = [next(file).strip() for _ in range(10)]
except FileNotFoundError:
    first_10_lines = "File not found."

first_10_lines


['[{"id": "12", "vector": [0.033237554132938385, -0.0028929722029715776, 0.00493069039657712, -0.025750312954187393, 0.02003820240497589, -0.022075528278946877, -0.020837847143411636, 0.03522766754031181, 0.04459790512919426, -0.03130921348929405, 0.027388738468289375, 0.015430128201842308, -0.035958025604486465, -0.038402918726205826, -0.014430449344217777, -0.01649686135351658, 0.013430445455014706, 0.03244579955935478, -0.0039698961190879345, -0.0028539339546114206, 0.031951747834682465, -0.0010826194193214178, -0.03223096579313278, -0.029852375388145447, -0.030756385996937752, 0.008703015744686127, -0.03148563206195831, -0.028573080897331238, -0.049125563353300095, -0.029775945469737053, -0.028770687058568, 0.01664712466299534, -0.03077717497944832, -0.04647235944867134, 0.008931194432079792, 0.05350237339735031, 0.03654354438185692, 0.030525481328368187, -0.03685972839593887, 0.02815299481153488, -0.04306894913315773, 0.06289734691381454, -0.0426960363984108, -0.045395366847515106

In [6]:
import json
import tqdm
from datasets import load_dataset

# Параметры
BATCH_SIZE = 1024

# Загрузка датасета
ds = load_dataset("wikimedia/wikipedia", "20231101.en")

# Инициализация временных списков для батчей
batch_ids = []
batch_payloads = []
batch_vectors = []

# Чтение большого файла построчно и загрузка в базу данных
with open('C:/Users/Profe/OneDrive/Документы/embeddings_part1.json', 'r') as file:
    for index, line in tqdm.tqdm(enumerate(file)):
        # Парсинг JSON-строки
        string = line.strip().rstrip(",").lstrip('[')
        if not string:
            break
        entry = json.loads(string)
        data = ds['train'][index]

        # Добавляем данные в текущий батч
        batch_ids.append(int(10e9) + int(entry['id']))
        batch_payloads.append({
            "text": data['text'],
            "my_metadata": {
                "source": data['url'],
                "title": data['title']
            }
        })
        batch_vectors.append(entry["vector"])

        # Если батч заполнен, делаем загрузку в базу данных
        if (index + 1) % BATCH_SIZE == 0:
            client.upsert(
                collection_name=COLLECTION_NAME,
                points=models.Batch(
                    ids=batch_ids,
                    payloads=batch_payloads,
                    vectors=batch_vectors,
                ),
            )
            # Очищаем временные списки для следующего батча
            batch_ids.clear()
            batch_payloads.clear()
            batch_vectors.clear()

    # Загрузка оставшихся данных (если общее количество записей не кратно BATCH_SIZE)
    if batch_ids:
        client.upsert(
            collection_name=COLLECTION_NAME,
            points=models.Batch(
                ids=batch_ids,
                payloads=batch_payloads,
                vectors=batch_vectors,

            ),
        )


404479it [1:19:12, 85.11it/s] 


KeyboardInterrupt: 

In [5]:
import json
import tqdm
from datasets import load_dataset

# Параметры
BATCH_SIZE = 1024

# Загрузка датасета
#ds = load_dataset("wikimedia/wikipedia", "20231101.en")

# Инициализация временных списков для батчей
batch_ids = []
batch_payloads = []
batch_vectors = []

# Чтение большого файла построчно и загрузка в базу данных
with open('D:\embeddings_part1_text512.json', 'r') as file:
    for index, line in tqdm.tqdm(enumerate(file)):
        # Парсинг JSON-строки
        string = line.strip().rstrip(",").lstrip('[')
        if not string:
            break
        entry = json.loads(string)
        #data = ds['train'][index]

        # Добавляем данные в текущий батч
        batch_ids.append(int(10e9) + int(entry['id']))
        '''
        batch_payloads.append({
            "text": data['text'],
            "my_metadata": {
                "source": data['url'],
                "title": data['title']
            }
        })
        '''
        batch_vectors.append(entry["vector"])

        # Если батч заполнен, делаем загрузку в базу данных
        if (index + 1) % BATCH_SIZE == 0:
            client.upsert(
                collection_name=COLLECTION_NAME,
                points=[
                    models.PointStruct(
                        id=batch_ids[i],
                        vector=batch_vectors[i]
                    )
                    for i in range(BATCH_SIZE)
                ]
            )
            # Очищаем временные списки для следующего батча
            batch_ids.clear()
            batch_payloads.clear()
            batch_vectors.clear()

    # Загрузка оставшихся данных (если общее количество записей не кратно BATCH_SIZE)
    if batch_ids:
        client.upsert(
            collection_name=COLLECTION_NAME,
            points=models.Batch(
                ids=batch_ids,
                #payloads=batch_payloads,
                vectors=batch_vectors,

            ),
        )


3203072it [11:35:39, 76.74it/s] 


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [6]:
if batch_ids:
    client.upsert(
        collection_name=COLLECTION_NAME,
        points=models.Batch(
            ids=batch_ids,
            #payloads=batch_payloads,
            vectors=batch_vectors,

        ),
    )

In [8]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name='intfloat/multilingual-e5-large',
    model_kwargs={"device": "cpu"},
    cache_folder='cache'
)

C:\Users\Profe\AppData\Local\Temp\ipykernel_6016\3294939613.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [9]:
del batch_ids
del batch_payloads
del batch_vectors
del ds

In [11]:
client.query_points(
        collection_name=COLLECTION_NAME,
        search_params=models.SearchParams(hnsw_ef=128, exact=False),
        query=embeddings.embed_query('query: ' + 'What do you know about volleyball?'),
        limit=5,
        with_vectors=True,
        with_payload=False
    )

In [ ]:
import json
import tqdm
from datasets import load_dataset

# Параметры
BATCH_SIZE = 1024

# Загрузка датасета
#ds = load_dataset("wikimedia/wikipedia", "20231101.en")

# Инициализация временных списков для батчей
batch_ids = []
batch_payloads = []
batch_vectors = []

# Чтение большого файла построчно и загрузка в базу данных
with open('D:\embeddings_part2_text512.json', 'r') as file:
    for index, line in tqdm.tqdm(enumerate(file)):
        # Парсинг JSON-строки
        string = line.strip().rstrip(",").lstrip('[')
        if not string:
            break
        entry = json.loads(string)
        #data = ds['train'][index]

        # Добавляем данные в текущий батч
        batch_ids.append(int(10e9) + int(entry['id']))
        '''
        batch_payloads.append({
            "text": data['text'],
            "my_metadata": {
                "source": data['url'],
                "title": data['title']
            }
        })
        '''
        batch_vectors.append(entry["vector"])

        # Если батч заполнен, делаем загрузку в базу данных
        if (index + 1) % BATCH_SIZE == 0:
            client.upsert(
                collection_name=COLLECTION_NAME,
                points=[
                    models.PointStruct(
                        id=batch_ids[i],
                        vector=batch_vectors[i]
                    )
                    for i in range(BATCH_SIZE)
                ]
            )
            # Очищаем временные списки для следующего батча
            batch_ids.clear()
            batch_payloads.clear()
            batch_vectors.clear()

    # Загрузка оставшихся данных (если общее количество записей не кратно BATCH_SIZE)
    if batch_ids:
        client.upsert(
            collection_name=COLLECTION_NAME,
            points=models.Batch(
                ids=batch_ids,
                #payloads=batch_payloads,
                vectors=batch_vectors,

            ),
        )


In [36]:
import json
import tqdm
from datasets import load_dataset

# Параметры
BATCH_SIZE = 1000

# Загрузка датасета
ds = load_dataset("wikimedia/wikipedia", "20231101.en")

# Инициализация временных списков для батчей
batch_ids = []
batch_payloads = []
batch_vectors = []

# Чтение большого файла построчно и загрузка в базу данных
with open('C:/Users/Profe/OneDrive/Документы/embeddings2.json', 'r') as file:
    for index, line in tqdm.tqdm(enumerate(file)):
        # Парсинг JSON-строки
        print(line)
        continue
        entry = json.loads(line.strip().rstrip(",").lstrip('['))
        data = ds['train'][index]

        # Добавляем данные в текущий батч
        batch_ids.append(int(10e9) + index)
        batch_payloads.append({
            "text": data['text'],
            "my_metadata": {
                "source": data['url'],
                "title": data['title']
            }
        })
        batch_vectors.append(entry["vector"])

        # Если батч заполнен, делаем загрузку в базу данных
        if (index + 1) % BATCH_SIZE == 0:
            client.upsert(
                collection_name=COLLECTION_NAME,
                points=models.Batch(
                    ids=batch_ids,
                    payloads=batch_payloads,
                    vectors=batch_vectors,
                ),
            )
            # Очищаем временные списки для следующего батча
            batch_ids.clear()
            batch_payloads.clear()
            batch_vectors.clear()

    # Загрузка оставшихся данных (если общее количество записей не кратно BATCH_SIZE)
    if batch_ids:
        client.upsert(
            collection_name=COLLECTION_NAME,
            points=models.Batch(
                ids=batch_ids,
                payloads=batch_payloads,
                vectors=batch_vectors,

            ),
        )


In [39]:
from qdrant_client import QdrantClient

# Получение списка всех коллекций
collections = client.get_collections().collections

# Удаление всех коллекций
for collection in collections:
    client.delete_collection(collection.name)
    print(f"Коллекция '{collection.name}' удалена")


In [31]:
len(ds['train']['id'])

6407814

In [36]:
import json
import tqdm
from datasets import load_dataset

# Параметры
BATCH_SIZE = 1000
LOWER_BOUND = 0
UPPER_BOUND = 2_000_000

# Загрузка датасета
ds = load_dataset("wikimedia/wikipedia", "20231101.en")

# Инициализация временных списков для батчей
batch_ids = []
batch_payloads = []
batch_vectors = []


# Чтение большого файла построчно и загрузка в базу данных
with open('C:/Users/Profe/OneDrive/Документы/embeddings.json.filepart', 'r') as file:
    for index, line in tqdm.tqdm(enumerate(file)):
        if index > UPPER_BOUND:
            break
        elif index < LOWER_BOUND:
            continue
        # Парсинг JSON-строки
        entry = json.loads(line.strip().rstrip(",").lstrip('['))
        data = ds['train'][index]

        # Добавляем данные в текущий батч
        batch_ids.append(int(10e9) + int(entry['id']))
        batch_payloads.append({
            "text": data['text'],
            "my_metadata": {
                "source": data['url'],
                "title": data['title']
            }
        })
        print(len(entry["vector"]))
        break
        if len(entry["vector"]) != 1024:
            break
        batch_vectors.append(entry["vector"])

        # Если батч заполнен, делаем загрузку в базу данных
        if (index + 1) % BATCH_SIZE == 0:
            client.upsert(
                collection_name=COLLECTION_NAME,
                points=models.Batch(
                    ids=batch_ids,
                    payloads=batch_payloads,
                    vectors=batch_vectors,
                ),
            )
            # Очищаем временные списки для следующего батча
            batch_ids.clear()
            batch_payloads.clear()
            batch_vectors.clear()

    # Загрузка оставшихся данных (если общее количество записей не кратно BATCH_SIZE)
    if batch_ids:
        client.upsert(
            collection_name=COLLECTION_NAME,
            points=models.Batch(
                ids=batch_ids,
                payloads=batch_payloads,
                vectors=batch_vectors,

            ),
        )

0it [00:00, ?it/s]

1024


IndexError: list index out of range

In [12]:
from fastembed import SparseTextEmbedding

documents = [
    "You should stay, study and sprint.",
    "History can only prepare us to be surprised yet again.",
]

model = SparseTextEmbedding(model_name="Qdrant/bm25")
embeddings = list(model.embed(documents))

# [
#     SparseEmbedding(
#         values=array([1.67419738, 1.67419738, 1.67419738, 1.67419738]),
#         indices=array([171321964, 1881538586, 150760872, 1932363795])),
#     SparseEmbedding(values=array(
#         [1.66973021, 1.66973021, 1.66973021, 1.66973021, 1.66973021]),
#                     indices=array([
#                         578407224, 1849833631, 1008800696, 2090661150,
#                         1117393019
#                     ]))
# ]


Fetching 29 files:   0%|          | 0/29 [00:00<?, ?it/s]c:\Users\Profe\.virtualenvs\code-uVaVH2Jt\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Profe\AppData\Local\Temp\fastembed_cache\models--Qdrant--bm25. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Fetching 29 files: 100%|██████████| 29/29 [00:

In [13]:
embeddings

[SparseEmbedding(values=array([1.67868852, 1.67868852, 1.67868852]), indices=array([1881538586,  150760872, 1932363795])),
 SparseEmbedding(values=array([1.66973021, 1.66973021, 1.66973021, 1.66973021, 1.66973021]), indices=array([ 733618285, 1849833631, 1008800696, 2090661150, 1117393019]))]

In [1]:
from langchain_community.embeddings import HuggingFaceEmbeddings